<a href="https://colab.research.google.com/github/Taiga10969/Lecture-Transformer/blob/main/copy_code_Transformer_03_Multi_Head_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 写経して理解するTransformer_03：Multi-Head Attention

Multi-Head Attentionは，Transformerモデルの重要な構成要素の1つである．<br>
Attention機構は，モデルが文字列を構成する各トークンに対して重要度の重み付けを行い，その情報を利用して出力を生成するために使用される．<br>
Multi-Head Attentionは，単一のAttentionヘッドではなく，複数のAttentionヘッドを使ってこの処理を行う．<br>
<br>
### **Multi-Head Attentionの概要**<br>

**Query，Key，Valueの射影**<br>
Multi-Head Attentionは，3つの異なるLinear層を持っており，元の入力の特徴量空間を異なる部分空間に射影する．<br>
具体的には，Attention機構で用いられるQuery，Key，Valueの3つに射影する．<br>
<br>
**Attentionスコアの計算**<br>
各Attentionヘッドでは，QueryとKeyの内積を計算してAttentionスコアを生成する．<br>
これにより，各Queryに対してKeyの各要素との関連度を表すスコアを獲得する．<br>
<br>
**重み付きのValueの取得**<br>
Attentionスコアはsoftmax関数を使用して正規化され，各Valueに重みが付けされる．<br>
この重みは，QueryとKeyの関連度を表し，その関連度に基づいてValueに重み付けする．<br>
<br>
**Multi-Headの結合**<br>
すべてのAttentionヘッドがこれらの重み付きValueを生成し，最後に各ヘッドで生成されたベクトルを結合する．<br>
これにより，多様な観点からの情報が網羅され，より豊かな表現が得られる．<br>
<br>
**最終的な出力の生成**<br>
結合された重み付きValueは，再度線形射影を通して最終的な出力を生成する．<br>
<br>
Multi-Head Attentionの利点は，異なる注意の表現を複数使用するところである．<br>
各ヘッドは異なる部分空間に射影されるため，異なる種類の情報をキャプチャし，モデルがより広範な文脈を捉えるのに役立つ．<br>
これにより，モデルの表現力が向上し，複雑な関係をモデル化できる．

## 必要ライブラリのインポート

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

## 【MultiHeadAttentionクラスの定義】

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)

        self.W_o = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        query = self.W_q(query)
        key = self.W_k(key)
        value = self.W_v(value)

        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        attention_score = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.depth, dtype=torch.float32))

        if mask is not None:
            attention_score += mask

        attention_weights = F.softmax(attention_score, dim=-1)
        attention = torch.matmul(attention_weights, value)
        attention = attention.permute(0, 2, 1, 3).contiguous()
        attention = attention.view(batch_size, -1, self.d_model)

        return self.W_o(attention)


## MultiHeadAttentionの挙動確認

In [3]:
# 疑似データ（Embedderによって1batch,7tokenが埋め込まれた，埋め込み次元数128のデータ）
input_data = torch.randn(1, 7, 128)

d_model = 128
num_heads = 4  # ヘッド数を4に設定

# Multi-Head Attentionモデルの作成
multi_head_attention = MultiHeadAttention(d_model, num_heads)

# テストデータをMulti-Head Attentionに流す
output = multi_head_attention(input_data, input_data, input_data)

# 出力のサイズを確認
print("output.shape:", output.shape)

output.shape: torch.Size([1, 7, 128])
